# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.40,84,78,9.54,TF,1722895778
1,1,thunder bay,48.4001,-89.3168,23.99,38,100,4.12,CA,1722895249
2,2,murzuq,25.9155,13.9184,32.09,13,0,5.11,LY,1722895780
3,3,papao,-17.7500,-149.5500,28.82,74,7,6.31,PF,1722895782
4,4,bredasdorp,-34.5322,20.0403,7.84,84,0,3.34,ZA,1722895783


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Criteria for ideal location
# - cloudiness less than or equal to 60%
# - Wind speed less than or equal to 5 m/s
# - Max temp less than or equal to 28 degrees celsius but greater than or equal to 21 degrees celsius

# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Cloudiness"]<= 60) & (city_data_df["Wind Speed"]<=5) & (city_data_df["Max Temp"]>=21) & (city_data_df["Max Temp"]<=28)]
ideal_city_df


# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,meizhou,24.2977,116.1072,25.27,96,53,0.23,CN,1722895794
22,22,beidaihehaibin,39.8222,119.5181,24.58,87,6,1.87,CN,1722895802
25,25,vila franca do campo,37.7167,-25.4333,24.23,89,39,2.41,PT,1722895805
37,37,puerto ayora,-0.7393,-90.3518,23.98,80,60,4.57,EC,1722895818
54,54,whitehorse,60.7161,-135.0538,26.17,32,20,4.63,CA,1722895584


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.sample(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
261,rio claro,BR,-22.4114,-47.5614,36,
124,howard springs,AU,-12.4970,131.0470,88,
331,piranshahr,IR,36.7010,45.1413,32,
354,fayzabad,AF,37.1166,70.5800,34,
408,pontianak,ID,-0.0333,109.3333,100,
437,touba,CI,8.2833,-7.6833,97,
329,khash,IR,28.2211,61.2158,46,
140,itamaraca,BR,-7.7478,-34.8256,79,
417,unai,BR,-16.3575,-46.9061,38,
22,beidaihehaibin,CN,39.8222,119.5181,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "format":"json",
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row ["Lat"]
    longitude = row ["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
meizhou - nearest hotel: 7天梅州嘉得利店
beidaihehaibin - nearest hotel: 北戴河爱尚家临海酒店
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
whitehorse - nearest hotel: Town & Mountain Hotel
hawaiian paradise park - nearest hotel: No hotel found
kalamata - nearest hotel: Byzantium
san julian - nearest hotel: No hotel found
sadovoye - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
tete - nearest hotel: Zambezi Hotel
ponta delgada - nearest hotel: Hotel Talisman
vyazemskiy - nearest hotel: Гостиница
georgetown - nearest hotel: Page 63 hostel
holualoa - nearest hotel: Kona Hotel
howard springs - nearest hotel: Rydges Palmerston
itamaraca - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
kasese - nearest hotel: Hotel
sault ste. marie - nearest hotel: Holiday Inn
caorle - nearest hotel: Hotel Maxim
roma - nearest hotel: Wingate by Wyndham Rome
shahecheng - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
15,meizhou,CN,24.2977,116.1072,96,7天梅州嘉得利店
22,beidaihehaibin,CN,39.8222,119.5181,87,北戴河爱尚家临海酒店
25,vila franca do campo,PT,37.7167,-25.4333,89,Vinha d'Areia Beach Hotel
37,puerto ayora,EC,-0.7393,-90.3518,80,Hostal La Mirada De Solitario George
54,whitehorse,CA,60.7161,-135.0538,32,Town & Mountain Hotel
...,...,...,...,...,...,...
503,padang,ID,-0.9492,100.3543,88,losmen Surya
521,nurota,UZ,40.5614,65.6886,23,Nurota
525,corner brook,CA,48.9667,-57.9484,76,No hotel found
527,sinan,KR,34.8262,126.1086,87,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.8,
    scale = 1.1,
    color = "City",
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)